In [1]:
import findspark
findspark.init("/opt/cloudera/parcels/SPARK2-2.2.0.cloudera1-1.cdh5.12.0.p0.142354/lib/spark2/")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Redes_Neuronales').getOrCreate()
import pandas

/usr/lib/python2.7/site-packages/subprocess32.py:146: RuntimeWarning: The _posixsubprocess module is not being used. Child process reliability may suffer if your program uses threads.
  "program uses threads.", RuntimeWarning)


In [4]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.linalg import Vectors, SparseVector, DenseVector
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pandas.tools.plotting import scatter_matrix
%matplotlib inline 
import matplotlib.pyplot as plt, numpy as np


In [5]:
trainning = spark.read.csv("/user/epinedac/datasets/iris", header=True, nullValue="?", inferSchema=True)
trainning.show()

+------------+-----------+------------+-----------+-------+-----+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|Species|label|
+------------+-----------+------------+-----------+-------+-----+
|         5.1|        3.5|         1.4|        0.2| setosa|    1|
|         4.9|        3.0|         1.4|        0.2| setosa|    1|
|         4.7|        3.2|         1.3|        0.2| setosa|    1|
|         4.6|        3.1|         1.5|        0.2| setosa|    1|
|         5.0|        3.6|         1.4|        0.2| setosa|    1|
|         5.4|        3.9|         1.7|        0.4| setosa|    1|
|         4.6|        3.4|         1.4|        0.3| setosa|    1|
|         5.0|        3.4|         1.5|        0.2| setosa|    1|
|         4.4|        2.9|         1.4|        0.2| setosa|    1|
|         4.9|        3.1|         1.5|        0.1| setosa|    1|
|         5.4|        3.7|         1.5|        0.2| setosa|    1|
|         4.8|        3.4|         1.6|        0.2| setosa|    1|
|         

In [6]:
assembler = VectorAssembler(inputCols=["Sepal_Length", "Sepal_Width", "Petal_Length", "Petal_Width"], outputCol="features")
assem_data = assembler.transform(trainning)

train_scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
train_scaler_model = train_scaler.fit(assem_data)
scaled_data_train = train_scaler_model.transform(assem_data)

In [7]:
splits = scaled_data_train.randomSplit([0.7, 0.3], 1234)
train = splits[0]
test = splits[1]

# Creando modelo asignando las capas 

layers = [4, 1, 4]

trainerassem = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
trainedModel = trainerassem.fit(train)

In [10]:
print trainedModel.weights

[-1.3254788823165669,-8.461648570404304,14.003872022704536,4.898695552907848,-1.7813087757716815,-77.3949743183719,10.599971536648164,33.02190120410698,33.86170247977203,-76.83272336658241,33.31416843077716,22.042447563851976,21.120414098429286]


In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


result = trainedModel.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
evaluatorF1 = MulticlassClassificationEvaluator(metricName="f1")

print("Accuracy = " + str(evaluator.evaluate(predictionAndLabels)))
print("F1 Score = " + str(evaluatorF1.evaluate(predictionAndLabels)))

Accuracy = 0.625
F1 Score = 0.509615384615


In [13]:
#codigo para obtener la matriz de confusión
result.createOrReplaceTempView("tabla_rna")

In [14]:
spark.sql("""
            select label as real, prediction, count(1) as cantidad
              from tabla_rna
              group by label, prediction
              order by 1, 2 asc
          """).show()

+----+----------+--------+
|real|prediction|cantidad|
+----+----------+--------+
|   1|       1.0|      13|
|   2|       2.0|      12|
|   3|       2.0|      15|
+----+----------+--------+



In [ ]:
spark.stop()